In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'

In [2]:
file_name = 'normalized_minmax_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
print(num_user, len(svd))

953 147697641


In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,0.420160,1
1,0,15,0.381676,1
2,0,22,0.372974,1
3,0,28,0.389718,1
4,0,33,0.397697,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
70,70,17557,2980.75
1102,1102,6926,1484.25
83,83,6571,1796.75
13496,13496,6299,1669.75
952,952,6234,1527.50
2519,2519,6001,1772.25
4008,4008,5676,1008.25
5521,5521,5658,1727.25
3900,3900,5627,1475.50
2241,2241,5571,1183.75


In [7]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
70,70,17557,5.000000
1102,1102,6926,1.972433
83,83,6571,1.871333
13496,13496,6299,1.793871
952,952,6234,1.775360
2519,2519,6001,1.709005
4008,4008,5676,1.616449
5521,5521,5658,1.611323
3900,3900,5627,1.602495
2241,2241,5571,1.586547


In [9]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [11]:
for lambda_ in tqdm(lambdas):
    top_n_size = 20
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    print(avg_precisions)
#     print(sum(avg_precisions) / len(avg_precisions))
#     print('----------------------------------------') 

[0.18227146814404435, 0.21415094339622634, 0.23402061855670098, 0.2833333333333333]
[0.18393351800554023, 0.2141509433962264, 0.23505154639175255, 0.2840909090909091]
[0.1893351800554018, 0.2160377358490566, 0.23814432989690712, 0.28560606060606064]
[0.19445983379501403, 0.22169811320754726, 0.24587628865979375, 0.2916666666666667]
[0.2030470914127426, 0.22452830188679238, 0.25103092783505143, 0.2977272727272728]
[0.21080332409972322, 0.23113207547169792, 0.2530927835051545, 0.3022727272727273]
[0.22160664819944634, 0.24276729559748406, 0.26546391752577314, 0.31287878787878776]
[0.23753462603878145, 0.25628930817610057, 0.27680412371134017, 0.3219696969696969]
[0.2674515235457065, 0.2798742138364779, 0.2912371134020619, 0.33560606060606063]
[0.30457063711911375, 0.31698113207547185, 0.3226804123711339, 0.3590909090909091]
[0.2677285318559559, 0.2981132075471699, 0.30824742268041233, 0.35757575757575755]



In [ ]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        